In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit


In [2]:
data = pd.read_csv("../data/HMIS_DATA_CORRECTED_17_21/mh_dist17_21_with_IDs_date_correction.csv")
data = data[(data['indicator_type'] == 'Total [(A+B) or (C+D)]')]
data['date'] = pd.to_datetime(data['date'])
data = data.set_index('date')
data.index = pd.DatetimeIndex(data.index)

In [3]:
def ridge_regression_lags_only(
    series,
    district_name,
    max_lags=3,
    differencing=True,
    alpha=1.0,
    use_cv=False,
    alphas=np.logspace(-3, 2, 20),  # Only used if use_cv=True
    cv=5,  # Only used if use_cv=True
    random_state=42
):
    """
    Ridge regression (L2 regularization) for time series forecasting with lag features only.
    Supports optional cross-validation for alpha selection.
    """
    # Create directory structure
    os.makedirs('RidgeRegression', exist_ok=True)
    
    # 1. Stationarity and Differencing
    original_series = series.copy()
    d = 0
    if differencing:
        adf_result = adfuller(series.dropna())
        if adf_result[1] > 0.05:
            d = 1
            series = series.diff().dropna()

    # 2. Feature Engineering - Lag features only
    df = pd.DataFrame({'y': series.astype(float)})
    for lag in range(1, max_lags + 1):
        df[f'lag_{lag}'] = df['y'].shift(lag).astype(float)
    df = df.dropna()

    # 3. Train-Test Split (time-based)
    train_size = int(len(df) * 0.8)
    train = df.iloc[:train_size]
    test = df.iloc[train_size:]

    X_train = train.drop(columns=['y'])
    y_train = train['y']
    X_test = test.drop(columns=['y'])
    y_test = test['y']

    # 4. Model Training with Ridge or RidgeCV
    if use_cv:
        model = RidgeCV(alphas=alphas, cv=TimeSeriesSplit(n_splits=5), scoring='neg_mean_squared_error')
        model.fit(X_train, y_train)
        best_alpha = model.alpha_
    else:
        model = Ridge(alpha=alpha, random_state=random_state, max_iter=10000)
        model.fit(X_train, y_train)
        best_alpha = alpha

    # 5. Forecasting
    pred_test = model.predict(X_test)

    # 6. Inverse Differencing if applied
    if d == 1:
        last_train_value = original_series.iloc[len(original_series) - len(test) - 1]
        pred_test = np.cumsum(pred_test) + last_train_value
        y_test = original_series.iloc[-len(test):]
    
    # 7. Create forecast DataFrame
    forecast_df = pd.DataFrame({
        'district': district_name,
        'date': y_test.index,
        'actual': y_test.values,
        'forecast': pred_test
    })

    # 8. Save forecasts to CSV
    forecast_csv_path = os.path.join('RidgeRegression', 'ridge_forecasts_lags_only.csv')
    write_header = not os.path.exists(forecast_csv_path)
    forecast_df.to_csv(forecast_csv_path, mode='a', header=write_header, index=False)

    # 9. Calculate metrics
    rmse = np.sqrt(mean_squared_error(forecast_df['actual'], forecast_df['forecast']))
    r_squared = model.score(X_train, y_train)
    metrics_df = pd.DataFrame([{
        'district': district_name,
        'rmse': rmse,
        'r_squared': r_squared,
        'coefficients': model.coef_.tolist(),
        'intercept': model.intercept_,
        'best_alpha': best_alpha,
        'alphas_tried': alphas.tolist() if use_cv else [alpha],
        'differencing': d
    }])

    metrics_csv_path = os.path.join('RidgeRegression', 'ridge_metrics_lags_only.csv')
    write_header_metrics = not os.path.exists(metrics_csv_path)
    metrics_df.to_csv(metrics_csv_path, mode='a', header=write_header_metrics, index=False)

    # 10. Plotting
    plt.figure(figsize=(12, 6))
    plt.plot(original_series.index, original_series.values, label='Original Series')
    plt.plot(forecast_df['date'], forecast_df['forecast'], label='Forecast', color='darkorange')
    plt.title(
        f'Ridge Regression (Lags Only) for {district_name}\n'
        f'Alpha: {best_alpha:.4f} | RMSE: {rmse:.2f}, R²: {r_squared:.2f}'
    )
    plt.legend()
    plt.savefig(os.path.join('RidgeRegression', f'Ridge_forecast_lags_only_{district_name}.png'))
    plt.close()

    return {
        'district': district_name,
        'forecast_df': forecast_df,
        'metrics_df': metrics_df,
        'model_coefficients': model.coef_,
        'model_intercept': model.intercept_,
        'best_alpha': best_alpha,
        'alphas_tried': alphas if use_cv else [alpha]
    }

In [4]:
districts = data['district'].unique()
rmse_values = []

def run_for_each_district():
    results = {}
    
    for district in districts:
        district_data = data[data['district'] == district]
        ts = district_data["I8"].asfreq('MS')
        
        results = ridge_regression_lags_only(
            ts, 
            district, 
            max_lags=5, 
            use_cv=True, 
            alphas=np.logspace(-3, 2, 30), 
            cv=5
        )
        
    
        # Show results
        print("=== Metrics ===")
        print(results['metrics_df'])
        print("\n=== Forecast Data ===")
        print(results['forecast_df'].head())
    
    return results
run_for_each_district()

=== Metrics ===
     district         rmse  r_squared  \
0  AHMEDNAGAR  1364.808099   0.220637   

                                        coefficients  intercept  best_alpha  \
0  [-0.3504264408381702, -0.17094944794194308, -0...  119.11364       100.0   

                                        alphas_tried  differencing  
0  [0.001, 0.0014873521072935117, 0.0022122162910...             1  

=== Forecast Data ===
     district       date  actual     forecast
0  AHMEDNAGAR 2020-07-01    5982  6310.798179
1  AHMEDNAGAR 2020-08-01    5559  6368.520525
2  AHMEDNAGAR 2020-09-01    5903  7215.045148
3  AHMEDNAGAR 2020-10-01    5719  7447.627098
4  AHMEDNAGAR 2020-11-01    5427  7501.731083
=== Metrics ===
  district       rmse  r_squared  \
0    AKOLA  304.34163   0.153861   

                                        coefficients  intercept  best_alpha  \
0  [-0.30071380030727013, -0.12235356834531007, -...  36.727714       100.0   

                                        alphas_tried  dif

ValueError: Cannot have number of folds=6 greater than the number of samples=4.